# Variational Principle method

This notebook follows (Thijssen, 2013, p. 29). The variational method is a method for deducing upper bounds of energy eigenstates (usually used on the ground state). 

## Theory 

Consider a subset of Hilbert space whose energy expectation values are greater than some eigenstate's energy that we want to find. It can be shown that the min energy expectation value of any superposition of these states is greater than or equal to the eigenenergy we want (see (Thijssen, 2013, p. 30) for the full proof or (Griffiths, 2018, p. 327) for the simple case).

In the linear case this can be reformed into a matrix problem.

If the subset of states is $\{\ket{n}|n=1,2,3,...\}$ (this set doesn't have to be orthonormal), then a general ket takes the form:

$\ket{\phi} = \sum\limits_{i = 1}^{\infty}C_i\ket{i}$

$\implies \bra{\phi}\hat{H}\ket{\phi} = \dfrac{\sum\limits_{i,j = 1}^{\infty}C_i^{*}C_jH_{ij}}{\sum\limits_{i,j = 1}^{\infty}C_i^{*}C_jS_{ij}}$

where $H_{ij} = \bra{i}\hat{H}\ket{j}$ and $S_{ij} = \bra{i}\hat{S}\ket{j}$

Minimising $\bra{\phi}\hat{H}\ket{\phi}$ by calculus gives:

$\sum\limits_{j = 1}^{\infty}C_j(H_{ij} - E S_{ij})$ for all i

This can be rewritten as a matrix equation $\bf{H}\bf{C} = E \bf{S}\bf{C}$

## Linear variational method for the Hydrogen atom

In atomic units the time-independent Schrödinger equation is:

$(\frac{1}{2}\nabla^2 - \frac{1}{r})\psi(\vec{r}) = E\psi(\vec{r})$

Apply the linear variational principle method, where the guessed wavefunction is a linear superposition of:

$\psi_{\alpha_n}(\vec{r}) = e^{-\alpha_n r^2}$

where $\alpha \in \{ \alpha_1 = 13.00773, \alpha_2 = 1.962079, \alpha_3 = 0.444529, \alpha_4 = 0.1219492 \}$

The corresponding matrix elements are:

$S_{nm} = \left( \frac{\pi}{\alpha_n + \alpha_m} \right)^{\frac{3}{2}}$

$H_{nm} = 3\frac{\alpha_n\alpha_m\pi^{\frac{3}{2}}}{(\alpha_n + \alpha_m)^{\frac{5}{2}}} - \frac{2\pi}{\alpha_n + \alpha_m}$

The following program implements this approach.

In [62]:
import numpy as np
from scipy.linalg import eigh
from scipy.optimize import fmin


In [63]:

#functions that calculate components 
def Sij(alpha_i,alpha_j):
    return (np.pi/(alpha_i + alpha_j))**1.5

def Hij(alpha_i,alpha_j):
    return 3*(alpha_i*alpha_j*(np.pi**1.5))/((alpha_i + alpha_j)**(5/2)) - 2*np.pi/(alpha_i + alpha_j)


#values of alpha
alphas1 = np.array([13.00773, 1.962079, 0.444529, 0.1219492])

def findHydrogenEnergies(alphas):
    
    #finding matrices as lists
    S = np.array([[Sij(a_i,a_j) for a_i in alphas] for a_j in alphas])
    H = np.array([[Hij(a_i,a_j) for a_i in alphas] for a_j in alphas])

    #find eigenvalues
    eigenvalues, eigenvectors = eigh(H,S)
    
    return eigenvalues

#print out
groundEnergy = findHydrogenEnergies(alphas1)[0]
print(f"Ground energy is {groundEnergy} hartree. The correct value is -0.5 hartree.")

Ground energy is -0.49927840566748466 hartree. The correct value is -0.5 hartree.


In [64]:
#minimisation (note should look at excited state of hydrogen, look for errors, graph?)

#Self-Consistent Molecular Orbital Methods. VI. Energy Optimized Gaussian Atomic Orbitals
#"Direct Search" Solution of Numerical and Statistical Problems*
#implement method described here to find alphas
#also look at alternative methods described in the paper above


print(findHydrogenEnergies(np.array([13, 1, 0.1, 2]))[0])
#constraints on alpha are that can't be 0 and alphas can't be equal
#steepest decent method didn't work as there are implict constraints in this problem (eg alphas can't be negative)

-0.46918228822584507


## Helium atom 

Via the Born-Oppenheimer, the Helium atom Hamiltonian is:

$\hat{H} = -\frac{1}{2}\nabla_1^2 -\frac{1}{2}\nabla_2^2 + \frac{1}{|\vec{r_1} - \vec{r_2}|}  - \frac{2}{r_1} - \frac{2}{r_2}$

As the ground state energy is in a singlet spin state, the position part of the wavefunction is $\psi (\vec{r_1},\vec{r_2}) = \phi (\vec{r_1})\phi (\vec{r_2})$. Assuming that $\phi (\vec{r})$ is normalised, it can be shown that (Thijssen, 2013, p. 46):

$(-\frac{1}{2}\nabla_1^2 - \frac{2}{r_1} + \int dV_2 \frac{|\phi{\vec{r_2}}|^2}{|\vec{r_1} - \vec{r_2}|})\phi(\vec{r_1}) = E\phi(\vec{r_1})$

This equation cannot be solved exactly. However, we can get an upper bound using the linear variational method described above. Using the same $\psi_{\alpha_n}(\vec{r})$ with different values of $\alpha$, this can be reformulated into a matrix problem:

$\sum\limits_{q}(h_{pq} + \sum\limits_{r,s}C_s C_r Q_{prqs})C_q = E\sum\limits_{q} S_{pq}C_q$

where

$h_{pq} = \bra{\psi_{\alpha_p}}-\frac{1}{2}\nabla^2 - \frac{2}{r}\ket{\psi_{\alpha_p}}$

$Q_{prqs} = \int dV_1 dV_2 \frac{\psi_{\alpha_p}(\vec{r_1})\psi_{\alpha_r}(\vec{r_2})\psi_{\alpha_q}(\vec{r_1})\psi_{\alpha_s}(\vec{r_2})}{|\vec{r_1}-\vec{r_2}|}$

$S_{pq} = \bra{\psi_{\alpha_p}}\ket{\psi_{\alpha_q}}$

This can be solved by guessing the $C$ values, plugging them into the $Q$ part of the expression and solving the eigenvalue equation produced. The process can then be repeated recursively with the $C$ s found until the error is below some specified value (works because the ground state energy is a stable fixed point under this map).

The ground state energy can then be deduced from:

$E_G = 2\sum\limits_{p,q}C_p C_q h_{pq} +\sum\limits_{p,q,r,s} Q_{prqs}C_p C_q C_s C_r$

This process is implemented below.

In [65]:
#simulation parameters
alphas2 = [0.298073, 1.242567, 5.782948, 38.47497]


In [66]:
#functions to calculate matrix elements 

def Qijkl(alpha_i,alpha_j,alpha_k,alpha_l):
    """
    Calculates the components of the Q matrix.
    """
    denominator = (alpha_i + alpha_k)*(alpha_j + alpha_l)*((alpha_i + alpha_j + alpha_k + alpha_l)**0.5)
    return 2*(np.pi**2.5)/denominator

def hij(alpha_i,alpha_j):
    """
    Finds components of h matrix
    """
    return (3*alpha_i*alpha_j*(np.pi**1.5))/((alpha_i + alpha_j)**2.5) -  4*np.pi/(alpha_i + alpha_j)



In [67]:
#find matricies
h = np.zeros((4, 4))
S2 = np.zeros((4, 4))
Q = np.zeros((4, 4, 4, 4))

for i in range(4):
    for j in range(4):
        h[i,j] = hij(alphas2[i],alphas2[j])
        S2[i,j] = Sij(alphas2[i], alphas2[j])
        for k in range(4):
            for l in range(4):
                Q[i, j, k, l]= Qijkl(alphas2[i],alphas2[j],alphas2[k],alphas2[l])



In [68]:

#normalisation function 
def normalise(C):
    """
    Normalises C (remember that the set doesn't have to be orthonomal so this is not trivial)
    """
    innerProduct = 0
    for i,c_i in enumerate(C):
        for j,c_j in enumerate(C):
            innerProduct += c_i*c_j*S2[i][j]

    return C/(innerProduct**0.5)  



In [69]:
#intial guess
CGuess = normalise(np.array([1, 1, 1, 1])) 
EGuess = 0

#run simulation
def iterateC(C,EPrev):
    """
    Performs one iteration of the recursive cycle   
    """
    #finds F matrix
    F = h + np.einsum('ikjl,k,l', Q, C, C) 

    #finds new C values
    eigenvalues, eignestates = eigh(F,S2,eigvals_only=False)
    CNew = normalise(eignestates[:,0])

    #find E
    groundEnergyFound =  2*np.einsum('ij,i,j', h, CNew, CNew) + np.einsum('ikjl,i,j,k,l', Q, CNew, CNew, CNew, CNew)
    
    #iterates agian if difference between previous iteration is not less than max error
    if abs(groundEnergyFound - EPrev) < 1E-14:
        return groundEnergyFound 
    else:
        return iterateC(CNew,groundEnergyFound)

print(f"Ground energy is {iterateC(CGuess,EGuess)} hartree. The correct value is -2.903 hartree.")

Ground energy is -2.8551603823702516 hartree. The correct value is -2.903 hartree.


## References

Thijseen J., 2013. Computational Physics. Cambridge: Cambridge University Press

Griffiths D., 2018. Introduction To Quantum Mechanics.  Cambridge: Cambridge University Press